In [17]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import torchvision
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import numpy as np

In [18]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 42
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

## Configure loaders

In [19]:
data_dir = 'data/'

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(
      data_dir,
      train=True,
      download=True,
      transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))
                             ])),
      batch_size=batch_size_train,
      shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                                 ])),
  batch_size=batch_size_test, shuffle=True)
    
type(test_loader)

torch.utils.data.dataloader.DataLoader

## Create network model

In [20]:
class ImageClass(nn.Module):
    def __init__(self):
        super(ImageClass, self).__init__()
        self.linear1 = nn.Linear(1 * 28 * 28, 20)
        self.linear2 = nn.Linear(20, 20)
        self.linear3 = nn.Linear(20, 10)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        return F.log_softmax(x)

In [21]:
#Network model
model = ImageClass()

#check if gpu is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#Stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [22]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [31]:
def train(epoch):

  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    #move tensors to device
    data = data.to(device)
    target = target.to(device)
    #set parameters gradients to 0
    optimizer.zero_grad()
    
    #runs model with data and returns result in output
    #here data contains a 64 * 1 * 28 * 28 tensor, 64 is batch size
    output = model(data.reshape(-1, 1 * 28 * 28))
    
    #Use negative log likelihood loss.
    loss = F.nll_loss(output, target)
    
    #with this gradients are calculated
    loss.backward()
    
    #update gradients
    optimizer.step()
    
    #Display iteration statistics
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [32]:
def test():
  #Sets the module in evaluation mode
  model.eval()
  test_loss = 0
  correct = 0
    
  #dont update dynamic computation graph
  with torch.no_grad():
        #for every example in test
        for data, target in test_loader:
            #move tensors to device
            data = data.to(device)
            target = target.to(device)
            #evaluate the model
            output = model(data.reshape(-1, 1 * 28 * 28).to(device))
            #acumulate the loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [33]:
#test
#n_epochs = 1
#Check accuaracy before training
test()
tic = time.perf_counter()

for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
    
toc = time.perf_counter()

print(f"Time elapsed: {toc - tic:0.4f} seconds")

<ipython-input-20-8e9cecab2ba0>:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.3068, Accuracy: 1010/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305094
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.265162
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.265359
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.236372
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.130611
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.216084
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.199501
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.149719
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.026531
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.942366
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.882073
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.806234
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.809833
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.692557
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.647854
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.719519
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.699159
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.376281
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.710159
Train Epoch: 1 [12160

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.137003
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.249851
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.215686
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.167852
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.454760
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.331913
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.263078
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.403475
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.263298
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.517119
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.335477
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.150941
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.373633
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.155047
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.295822
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.232076
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.304582
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.467383
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.239386
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.253247


## Todo

- Try to enhance performance by adding more linear layers

## Notes

- Based on: https://nextjournal.com/gkoehler/pytorch-mnist